In [54]:
## JSON 테스트
import requests

url = "http://data.ex.co.kr/openapi/trtm/realUnitTrtm?key=1308370912&type=json&iStartUnitCode=101&iEndUnitCode=103"
req = requests.get(url)
print(req)

<Response [200]>


In [55]:
%%timeit

# JSON 데이터를 pandas df으로 가공
import pandas as pd

json_df = req.json()
df = pd.DataFrame(json_df['realUnitTrtmVO'])
df

722 µs ± 74.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [58]:
%%timeit

## XML 테스트_1
import requests
import lxml
from bs4 import BeautifulSoup

## XML 결괏값으로 데이터프레임으로 변환
url = "http://data.ex.co.kr/openapi/trtm/realUnitTrtm?key=1308370912&type=xml&iStartUnitCode=101&iEndUnitCode=103"
req = requests.get(url)
soup = BeautifulSoup(req.text, 'lxml') # XML 반환값이라서 lxml을 사용하여 파싱

stddate_list = soup.find_all('stddate')
timeAvg_list = soup.find_all('timeavg')

stddate = []
avgTime = []
for date, timeAvg in zip(stddate_list, timeAvg_list):
    stddate.append(date.get_text())
    avgTime.append(timeAvg.get_text())

df = pd.DataFrame({'date':stddate, 'avgTime':avgTime})
df

1.81 s ± 229 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [62]:
%%timeit

## XML 테스트_2

import pandas as pd
import xml.etree.ElementTree as ET
import requests
url = "http://data.ex.co.kr/openapi/trtm/realUnitTrtm?key=1308370912&type=xml&iStartUnitCode=101&iEndUnitCode=103"
response = requests.get(url)
root = ET.fromstring(response.content)
data_1 = [
    {element.tag: element.text for element in realUnitTrtmVO}
    for realUnitTrtmVO in root.findall('.//realUnitTrtmVO')
]
pd.DataFrame(data_1)

5.11 s ± 1.1 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [103]:
%%timeit

# 공공데이터 포털 by me

import requests
import pandas as pd

url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?LAWD_CD=11110&DEAL_YMD=201512&serviceKey=o0CmhlAvqpjrYb4A%2BYeoTcpt5T9kN0rHF7L1Tcrw8Th1mDZaspp%2Bj0JaW9edrluemEotB58VFRjSoJKAqb88dQ%3D%3D'

df = pd.read_xml(url, xpath='.//item')

print(df)

      거래금액  거래유형  건축년도     년      도로명  도로명건물본번호코드  도로명건물부번호코드  도로명시군구코드  \
0   82,500   NaN  2008  2015    사직로8길           4           0     11110   
1   60,000   NaN  1981  2015  세종대로23길          47           0     11110   
2  130,000   NaN  2004  2015    경희궁2길           5           5     11110   
3  105,000   NaN  2004  2015    사직로8길          24           0     11110   
4  120,000   NaN  2003  2015    사직로8길          20           0     11110   
5   17,000   NaN  2014  2015      대학로          47           0     11110   
6   17,000   NaN  2014  2015      대학로          47           0     11110   
7   57,000   NaN  2006  2015   혜화로3가길          30           0     11110   
8   44,000   NaN  1995  2015     창경궁로         265           0     11110   
9   52,000   NaN  1995  2015     창경궁로         265           0     11110   

   도로명일련번호코드  도로명지상지하코드  ...   월   일        일련번호    전용면적  중개사소재지      지번  \
0          3          0  ...  12  10  11110-2203   94.51     NaN       9   
1          2          

In [ ]:
!pip install xmltodict

In [99]:
# 공공데이터 포털 by teacher

import requests
import pandas as pd 

serviceKey = 'o0CmhlAvqpjrYb4A%2BYeoTcpt5T9kN0rHF7L1Tcrw8Th1mDZaspp%2Bj0JaW9edrluemEotB58VFRjSoJKAqb88dQ%3D%3D'
pageNo = 1 # 페이지 번호
numOfRows = 1000 # 한 페이지 결과수
LAWD_CD = 11560 # 지역코드
DEAL_YMD = 202307 # 계약월
url = f'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey={serviceKey}&pageNo={pageNo}&numOfRows=10&LAWD_CD={LAWD_CD}&DEAL_YMD={DEAL_YMD}'
req = requests.get(url)
req

<Response [200]>

In [102]:
%%timeit

import xmltodict
import pandas as pd

data_dict = xmltodict.parse(req.text)
pd.DataFrame(data_dict['response']['body']['items']['item'])

2.72 ms ± 135 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [107]:
# 서울 열린데이터 광장 by teacher

import requests
import json
import pandas as pd
SERVICE_KEY = '564b7852646a686a34336f4f6c5571'
url = f'http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/json/tbLnOpendataRtmsV/1/1000/2023/11560'
req = requests.get(url)
content = req.json()
result = pd.DataFrame(content['tbLnOpendataRtmsV']['row'])
result

,ACC_YEAR,SGG_CD,SGG_NM,BJDONG_CD,BJDONG_NM,LAND_GBN,LAND_GBN_NM,BONBEON,BUBEON,BLDG_NM,...,OBJ_AMT,BLDG_AREA,TOT_AREA,FLOOR,RIGHT_GBN,CNTL_YMD,BUILD_YEAR,HOUSE_TYPE,REQ_GBN,RDEALER_LAWDNM
0,2023,11560,영등포구,10900,영등포동8가,1,대지,0051,0001,메이준,...,8000,17.44,25.56,6.0,,,2012,오피스텔,중개거래,서울 영등포구
1,2023,11560,영등포구,13200,신길동,1,대지,4934,0000,삼성래미안,...,75000,59.33,0.00,5.0,,,2001,아파트,중개거래,서울 영등포구
2,2023,11560,영등포구,12100,문래동3가,1,대지,0013,0000,문래해태,...,68000,84.69,0.00,13.0,,,1999,아파트,중개거래,서울 영등포구
3,2023,11560,영등포구,10300,영등포동2가,1,대지,0028,0118,그린나래,...,37500,56.51,63.16,5.0,,,2014,오피스텔,중개거래,서울 영등포구
4,2023,11560,영등포구,13300,대림동,,,,,,...,77000,216.88,111.20,0.0,,,1985,단독다가구,직거래,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023,11560,영등포구,13000,양평동6가,1,대지,0075,0012,선유도윤영빌B동,...,47650,46.02,27.22,2.0,,,2023,연립다세대,직거래,
996,2023,11560,영등포구,13000,양평동6가,1,대지,0075,0012,선유도윤영빌B동,...,47300,46.03,27.22,2.0,,,2023,연립다세대,직거래,
997,2023,11560,영등포구,13000,양평동6가,1,대지,0075,0012,선유도윤영빌B동,...,47950,46.03,27.22,3.0,,,2023,연립다세대,직거래,
998,2023,11560,영등포구,13000,양평동6가,1,대지,0075,0012,선유도윤영빌B동,...,47950,46.03,27.22,4.0,,,2023,연립다세대,직거래,


In [125]:
import requests
import json
import pandas as pd
import xmltodict
SERVICE_KEY = '564b7852646a686a34336f4f6c5571'

data = None
for j in range(1,13):
    url = f'http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/json/tbLnOpendataRtmsV/{1+((j-1)*1000)}/{j*1000}'
    # url = f'http://openapi.seoul.go.kr:8088/{service_key}/json/tbLnOpendataRentV/1/1000/2023/11560'
    print(url)
    req = requests.get(url)
    content = req.json()
    con = content['tbLnOpendataRtmsV']['row']
    result = pd.DataFrame(con)
    data = pd.concat([data, result])
data = data.reset_index(drop=True)
data.head()

http://openapi.seoul.go.kr:8088/564b7852646a686a34336f4f6c5571/json/tbLnOpendataRtmsV/1/1000
http://openapi.seoul.go.kr:8088/564b7852646a686a34336f4f6c5571/json/tbLnOpendataRtmsV/1001/2000
http://openapi.seoul.go.kr:8088/564b7852646a686a34336f4f6c5571/json/tbLnOpendataRtmsV/2001/3000
http://openapi.seoul.go.kr:8088/564b7852646a686a34336f4f6c5571/json/tbLnOpendataRtmsV/3001/4000
http://openapi.seoul.go.kr:8088/564b7852646a686a34336f4f6c5571/json/tbLnOpendataRtmsV/4001/5000
http://openapi.seoul.go.kr:8088/564b7852646a686a34336f4f6c5571/json/tbLnOpendataRtmsV/5001/6000
http://openapi.seoul.go.kr:8088/564b7852646a686a34336f4f6c5571/json/tbLnOpendataRtmsV/6001/7000
http://openapi.seoul.go.kr:8088/564b7852646a686a34336f4f6c5571/json/tbLnOpendataRtmsV/7001/8000
http://openapi.seoul.go.kr:8088/564b7852646a686a34336f4f6c5571/json/tbLnOpendataRtmsV/8001/9000
http://openapi.seoul.go.kr:8088/564b7852646a686a34336f4f6c5571/json/tbLnOpendataRtmsV/9001/10000
http://openapi.seoul.go.kr:8088/564b785264

,ACC_YEAR,SGG_CD,SGG_NM,BJDONG_CD,BJDONG_NM,LAND_GBN,LAND_GBN_NM,BONBEON,BUBEON,BLDG_NM,...,OBJ_AMT,BLDG_AREA,TOT_AREA,FLOOR,RIGHT_GBN,CNTL_YMD,BUILD_YEAR,HOUSE_TYPE,REQ_GBN,RDEALER_LAWDNM
0,2023,11710,송파구,10800,문정동,1,대지,0652,0004,힐스테이트에코문정,...,20500,20.79,32.68,4.0,,,2018,오피스텔,중개거래,서울 송파구
1,2023,11560,영등포구,10900,영등포동8가,1,대지,0051,0001,메이준,...,8000,17.44,25.56,6.0,,,2012,오피스텔,중개거래,서울 영등포구
2,2023,11380,은평구,10500,구산동,1,대지,0345,0020,솔빛엘르빌,...,22000,49.30,27.73,2.0,,,2012,연립다세대,중개거래,서울 은평구
3,2023,11620,관악구,10100,봉천동,1,대지,1708,0000,두산,...,80000,59.92,0.00,10.0,,,2000,아파트,중개거래,서울 관악구
4,2023,11500,강서구,10300,화곡동,1,대지,1071,0007,와이앤비,...,22500,25.38,15.19,3.0,,,2017,연립다세대,중개거래,서울 강서구


In [128]:
data.to_csv('result.csv', index=False)